# Étude d'une équation de convection scalaire

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [ ]:
import numpy             as np
import numpy.linalg      as alg
import matplotlib.pyplot as plt
import time
#
import flowdyn.mesh  as mesh
import flowdyn.modelphy.euler as euler
import flowdyn.modeldisc      as modeldisc
#import flowdyn.field as data
from flowdyn.xnum        import *
from flowdyn.integration import *
import ipywidgets as pyw
%matplotlib notebook


## Définition des maillages, du modèle physique et solution initiales


In [ ]:
#mgmesh  = mesh.refinedmesh(ncell=100, length=1., ratio=2.)
mymodel = euler.model()
print(mymodel.list_bc())

## Calcul et comparaison de spectre



In [ ]:
w_M    = pyw.FloatSlider(value=.2, min=0, max=2,  step=.05, description="$M_0$")
w_nx   = pyw.IntSlider(value=50, min=2, max=500, step=2, description="$N_x$")
w_num  = pyw.Dropdown(description='Num. scheme:',
            options={'extrapol1/upwind': extrapol1(),
                     'extrapol2': extrapol2(),
                     'extrapol3': extrapol3()})
w_eps  = pyw.FloatLogSlider(value=1e-4, base=10, min=-10, max=0, step=.1, description='$\\varepsilon$', readout_format='3.1e')
w_vpnb = pyw.Checkbox(value=False, description="eigenvalue label") 
w_bc   = pyw.Checkbox(value=False, description="BC periodic") 
w_xr   = pyw.FloatRangeSlider(value=[-1., 0.1], min=-50., max=10.0, step=.01,
            description='$x$ range', readout=True, readout_format='.1f')
w_yr   = pyw.FloatRangeSlider(value=[-2., 2.], min=-100, max=100.0, step=.01,
            description='$y$ range', readout=True, readout_format='.1f', orientation='vertical')
w_trigger = pyw.Checkbox(value=False)

w_nummode = pyw.IntText(value=1, min=0, description='mode number:', continuous_update=False)
#w_nummode = pyw.IntSlider(value=1, min=0, step=1, description='mode number:', continuous_update=False)
w_var     = pyw.RadioButtons(description='variable set',
            options={'conservative': 'cons', 'primitive': 'prim'})
w_comp    = pyw.RadioButtons(description='complex',
            options={'real / imaginary': 'reimag', 'norm / angle': 'absang'})

def plot_spectra(mach, npts, modebc, num, epsdiff):
    global val, vec, lmesh
    rttot = (1.+.2*mach**2) / 1.4
    ptot  = (1.+.2*mach**2)**3.5
    lmesh = mesh.unimesh(ncell=npts, length=1.)
    #num   = extrapol3()
    bcp = { 'type': 'per' }
    bcL = { 'type': 'insub',  'ptot': ptot, 'rttot': rttot }
    bcR = { 'type': 'outsub', 'p': 1. }

    if modebc:
        rhs  = modeldisc.fvm(mymodel, lmesh, num, bcp, bcp)
    else:
        rhs  = modeldisc.fvm(mymodel, lmesh, num, bcL, bcR)

    finit = rhs.fdata(mymodel.prim2cons([1.4, mach, 1. ])) # rho, u, p

    solver = implicit(lmesh, rhs)
    jac    = solver.calc_jacobian(finit, epsdiff)
    val, vec = alg.eig(jac)
    
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ax.scatter(val.real, val.imag, marker='o')
    ax.axhline(0)
    ax.axvline(0)

    #plt.legend(labels, loc='upper left',prop={'size':10})
    #plot_zoom([-.1*lmesh.ncell, .02*lmesh.ncell], [-.5*lmesh.ncell, 1*lmesh.ncell])
    xmin, xmax = np.min(val.real), np.max(val.real)
    ymin, ymax = np.min(val.imag), np.max(val.imag)
    bord = .01
    w_xr.min = xmin - (xmax-xmin)*bord
    w_xr.max = xmax + (xmax-xmin)*bord
    w_yr.min = ymin - (ymax-ymin)*bord
    w_yr.max = ymax + (ymax-ymin)*bord
    w_vpnb.value = False
    w_nummode.max = val.size-1
    w_trigger.value = not w_trigger.value
    
def plot_zoom(xr, yr, vpnb=False, trigger=False):
    global val, vec
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ax.scatter(val.real, val.imag, marker='o')
    ax.set_xlim(xr[0], xr[1])
    ax.set_ylim(yr[0], yr[1])
    ax.axhline(0)
    ax.axvline(0)
    if vpnb:
        for i in range(val.size):
            if val[i].real > xr[0] and val[i].real < xr[1] and val[i].imag > yr[0] and val[i].imag < yr[1]:
                ax.annotate(i,(val[i].real, val[i].imag), annotation_clip=True)
    
def plot_mode(vartype, comprep, nummode):
    global vec, lmesh
    n = lmesh.ncell
    nvar=3
    print(nummode, n)
    fig, ax = plt.subplots(nvar, 2, figsize=(16,12))
    x = lmesh.centers()
    v = [None]*nvar
    for i in range(nvar):
        v[i] = vec[i::nvar,nummode]
    if comprep=='reimag':
        for i in range(nvar):
            ax[i][0].plot(x, v[i].real)
            ax[i][1].plot(x, v[i].imag)
    elif comprep=='absang':
        for i in range(nvar):
            ax[i][0].plot(x, np.abs(v[i]))
            ax[i][1].plot(x, np.angle(v[i], deg=True))
        

In [ ]:
out0 = pyw.interactive(plot_spectra, mach=w_M, npts=w_nx, modebc=w_bc, num=w_num, epsdiff=w_eps)
out0.children[-1].layout.height = '400px'
out1 = pyw.interactive(plot_zoom, xr=w_xr, yr=w_yr, vpnb=w_vpnb, trigger=w_trigger)
out1.children[-1].layout.height = '400px'

w_C00 = pyw.VBox([w_M, w_nx, w_num, w_eps, w_bc])
w_C01 = out0.children[-1]
w_C10 = pyw.VBox([pyw.Label('Zoom:'), w_vpnb, w_xr])
w_C11 = pyw.HBox([out1.children[-1],w_yr]) 

out2 = pyw.interactive(plot_mode, vartype=w_var, comprep=w_comp, nummode=w_nummode)
out2.children[-1].layout.height = '800px'
#def update_x_range(*args):
#    x_widget.max = 2.0 * y_widget.value
#y_widget.observe(update_x_range, 'value')

#w_T1 = pyw.HBox([w_C0, w_C1])
w_T1 = pyw.GridBox([w_C00, w_C10, w_C01, w_C11], layout=pyw.Layout(width='auto', grid_template_columns='50% 50%'))

w_T2 = pyw.VBox([w_nummode, pyw.HBox([w_var, w_comp]), out2.children[-1]])
output = pyw.Tab([w_T1, w_T2])
output.set_title(0, 'Spectra')
output.set_title(1, 'Modes')
output

In [ ]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style

In [ ]:
print('abc'=='ab')